In [1]:
%matplotlib inline

In [11]:
%run dog-breed-cnn.py
# Unzip the train and test zip file
archive_train = ZipFile("Data/test.zip", "r")
archive_test = ZipFile("Data/train.zip", "r")

# This line shows the 5 first image name of the train database
print(archive_train.namelist()[0:5])

# This line shows the number of images in the train database, 
# noted that we must remove the 1st value (column header)
print(len(archive_train.namelist()[:]) - 1)

SyntaxError: invalid syntax (<ipython-input-11-0d04216f0c00>, line 2)